In [30]:
import modin.pandas as pd
import time
import json
import os

In [33]:
dir_jobs = "C:\\Users\\eshad\\OneDrive\\Desktop\\TU Delft\\Thesis\\Web Scrape PoC\\Data\\3. Filtered preprocessed data with country\\all_data_with_countries\\"
dir_industries = "C:\\Users\\eshad\\OneDrive\\Desktop\\TU Delft\\Thesis\\Web Scrape PoC\\Data\\4. Filtered preprocessed data with industry\\"

In [34]:
# Function to extract company ID and LinkedIn link
def extract_company_info(company_detail):
    try:
        company_data = json.loads(company_detail.replace("'", '"'))
        if 'com.linkedin.voyager.deco.jobs.web.shared.WebCompactJobPostingCompany' in company_data:
            company_info = company_data['com.linkedin.voyager.deco.jobs.web.shared.WebCompactJobPostingCompany']
        elif 'com.linkedin.voyager.jobs.JobPostingCompanyName' in company_data:
            company_info = company_data['com.linkedin.voyager.jobs.JobPostingCompanyName']
        # print(company_info['company'])
        if 'company' in company_info:
            company_id = company_info['company'].split(':')[-1]
        elif 'companyResolutionResult' in company_info:
            company_id = company_info['companyResolutionResult']['entityUrn'].split(':')[-1]
        elif 'companyName' in company_info:
            company_id = company_info['companyName']
        return company_id
    except (KeyError, ValueError, TypeError) as e:
        return "no company ID"

In [35]:
dataframes = []
# Read all files in the directory
for file_name in os.listdir(dir_jobs):
    if file_name.startswith('all_data_with_countries_2'): 
        print(file_name)
        file_path = os.path.join(dir_jobs, file_name)
        tmp_df = pd.read_csv(file_path)
        dataframes.append(tmp_df)
df = pd.concat(dataframes)
print(len(df))
df = df.dropna(how='all')
df = df.drop_duplicates(keep='first')
print(len(df))
df.head()

all_data_with_countries_2016.csv
all_data_with_countries_2018.csv
61098
61098


,jobPostingId,companyDetails,title,description,workRemoteAllowed,formattedLocation,work_model,soc_code,country,year
0,114367533,{'com.linkedin.voyager.deco.jobs.web.shared.We...,Front End Developer,This is your opportunity to join Australia's o...,False,"Hawthorn, Victoria, Australia",NaN,213.0,Australia,2016
1,249165572,{'com.linkedin.voyager.deco.jobs.web.shared.We...,Manager - Cost Data Analyst,Position Description\n Cognizant is seeking a...,False,"Bengaluru, Karnataka, India",NaN,213.0,India,2016
2,249164672,{'com.linkedin.voyager.deco.jobs.web.shared.We...,Sr Agile/DevOps Coach,"As America's Un-carrier, T-Mobile US, Inc. (NA...",False,"Atlanta, GA",NaN,356.0,United States of America,2016
3,249164402,{'com.linkedin.voyager.deco.jobs.web.shared.We...,Data Scientist,"Rational is a full service, integrated agency ...",False,"Seattle, WA",NaN,242.0,United States of America,2016
4,249164246,{'com.linkedin.voyager.deco.jobs.web.shared.We...,Data Analyst - Cisco expereince a must,"Data Analyst - Finances & Operations Raleigh, ...",False,"Raleigh, NC",NaN,353.0,United States of America,2016


In [36]:
df['Company ID'] = df['companyDetails'].apply(lambda x: extract_company_info(x))
df.head()

,jobPostingId,companyDetails,title,description,workRemoteAllowed,formattedLocation,work_model,soc_code,country,year,Company ID
0,114367533,{'com.linkedin.voyager.deco.jobs.web.shared.We...,Front End Developer,This is your opportunity to join Australia's o...,False,"Hawthorn, Victoria, Australia",NaN,213.0,Australia,2016,2692410
1,249165572,{'com.linkedin.voyager.deco.jobs.web.shared.We...,Manager - Cost Data Analyst,Position Description\n Cognizant is seeking a...,False,"Bengaluru, Karnataka, India",NaN,213.0,India,2016,1680
2,249164672,{'com.linkedin.voyager.deco.jobs.web.shared.We...,Sr Agile/DevOps Coach,"As America's Un-carrier, T-Mobile US, Inc. (NA...",False,"Atlanta, GA",NaN,356.0,United States of America,2016,1392
3,249164402,{'com.linkedin.voyager.deco.jobs.web.shared.We...,Data Scientist,"Rational is a full service, integrated agency ...",False,"Seattle, WA",NaN,242.0,United States of America,2016,805290
4,249164246,{'com.linkedin.voyager.deco.jobs.web.shared.We...,Data Analyst - Cisco expereince a must,"Data Analyst - Finances & Operations Raleigh, ...",False,"Raleigh, NC",NaN,353.0,United States of America,2016,60921


In [37]:
print(len(df))

61098


In [58]:
industry_dataframes = []
# Read all files in the directory
for file_name in os.listdir(dir_industries):
    if file_name.startswith('industries_') and not file_name.startswith('industries_not'): 
        print(file_name)
        file_path = os.path.join(dir_industries, file_name)
        tmp_df = pd.read_csv(file_path, usecols = ['Company ID', 'Industry'])
        tmp_df = tmp_df[(tmp_df['Industry'] != "not found") | (tmp_df['Industry'] != "Industry Not Found")]
        industry_dataframes.append(tmp_df)
industry_df = pd.concat(industry_dataframes)
print(len(industry_df))
industry_df.head()

industries_0.csv
industries_1.csv
industries_2.csv
industries_3.csv
industries_4.csv
industries_5.csv
industries_6.csv
industries_7.csv
industries_8.csv
8459


,Company ID,Industry
0,2692410,Software Development
1,3075405,Software Development
2,3065197,Food and Beverage Manufacturing
3,13181517,Real Estate
4,468174,IT Services and IT Consulting


In [59]:
industry_df['Company ID'] = industry_df['Company ID'].apply(lambda x: str(x).strip().lower())
industry_df['Industry'] = industry_df['Industry'].apply(lambda x: str(x).strip().lower())
industry_df_no_dupes = industry_df.drop_duplicates(subset= ['Company ID', 'Industry'], keep='first', inplace=False).reset_index(drop=True)
print(len(industry_df_no_dupes))
industry_df_no_dupes.head()

8447


,Company ID,Industry
0,2692410,software development
1,3075405,software development
2,3065197,food and beverage manufacturing
3,13181517,real estate
4,468174,it services and it consulting


In [60]:
print(len(industry_df_no_dupes))
industry_df_no_dupes.head()

8447


,Company ID,Industry
0,2692410,software development
1,3075405,software development
2,3065197,food and beverage manufacturing
3,13181517,real estate
4,468174,it services and it consulting


In [61]:
industry_df_no_dupes_valid_data = industry_df_no_dupes[industry_df_no_dupes['Industry'] != "not found"]
industry_df_no_dupes_valid_data = industry_df_no_dupes_valid_data[industry_df_no_dupes_valid_data['Industry'] != "industry not found"]
print(len(industry_df_no_dupes_valid_data))
industry_df_no_dupes_valid_data.head()

4402


,Company ID,Industry
0,2692410,software development
1,3075405,software development
2,3065197,food and beverage manufacturing
3,13181517,real estate
4,468174,it services and it consulting


In [62]:
len(industry_df_no_dupes_valid_data[industry_df_no_dupes_valid_data['Industry'] == "not found"]), len(industry_df_no_dupes_valid_data[industry_df_no_dupes_valid_data['Industry'] == "industry not found"])

(0, 0)

In [63]:
industry_df_no_dupes_valid_data = industry_df_no_dupes_valid_data.drop_duplicates(subset= ['Company ID'], keep='last', inplace=False).reset_index(drop=True)
len(industry_df_no_dupes_valid_data)

4402

In [64]:
industry_df_no_dupes_valid_data['Company ID'].value_counts()

Company ID
100027     1
5038354    1
5053115    1
504840     1
5048039    1
          ..
23155      1
231711     1
2318430    1
2318649    1
zillow     1
Name: count, Length: 4402, dtype: int64

In [65]:
industry_df_no_dupes_valid_data.to_csv(f"{dir_industries}\\Industries and Codes\\industries_and_codes_batch_3.csv", index=False)

## Joining Code

In [67]:
industry_df_1 = pd.read_csv(os.path.join(dir_industries, 'Industries and Codes', 'industries_and_codes.csv'))
industry_df_2 = pd.read_csv(os.path.join(dir_industries, 'Industries and Codes', 'industries_and_codes_batch_2.csv'))
industry_df_3 = pd.read_csv(os.path.join(dir_industries, 'Industries and Codes', 'industries_and_codes_batch_3.csv'))
industry_df_no_dupes_valid_data = pd.concat([industry_df_1, industry_df_2, industry_df_3])
industry_df_no_dupes_valid_data = industry_df_no_dupes_valid_data[['Company ID', 'Industry']]
industry_df_no_dupes_valid_data = industry_df_no_dupes_valid_data.drop_duplicates(subset= ['Company ID', 'Industry'], keep='first', inplace=False).reset_index(drop=True)
print(len(industry_df_no_dupes_valid_data))
print(industry_df_no_dupes_valid_data.value_counts())
industry_df_no_dupes_valid_data.head()

66506
Company ID  Industry                     
1000        entertainment providers          1
5039143     financial services               1
503544      biotechnology research           1
503560      advertising services             1
5035753     it services and it consulting    1
                                            ..
21405       software development             1
2140522     human resources services         1
21407       biotechnology research           1
214071      staffing and recruiting          1
zype        financial services               1
Name: count, Length: 66505, dtype: int64


,Company ID,Industry
0,29023961,it services and it consulting
1,1313464,staffing and recruiting
2,223447,software development
3,67539665,software development
4,2388,financial services


In [68]:
df_with_industries = df.merge(industry_df_no_dupes_valid_data, how='left', on='Company ID').drop('Unnamed: 0', axis=0)
print(len(df_with_industries))
df_with_industries.head()

61098


,jobPostingId,companyDetails,title,description,workRemoteAllowed,formattedLocation,work_model,soc_code,country,year,Company ID,Industry
0,114367533,{'com.linkedin.voyager.deco.jobs.web.shared.We...,Front End Developer,This is your opportunity to join Australia's o...,False,"Hawthorn, Victoria, Australia",NaN,213.0,Australia,2016,2692410,software development
1,249165572,{'com.linkedin.voyager.deco.jobs.web.shared.We...,Manager - Cost Data Analyst,Position Description\n Cognizant is seeking a...,False,"Bengaluru, Karnataka, India",NaN,213.0,India,2016,1680,it services and it consulting
2,249164672,{'com.linkedin.voyager.deco.jobs.web.shared.We...,Sr Agile/DevOps Coach,"As America's Un-carrier, T-Mobile US, Inc. (NA...",False,"Atlanta, GA",NaN,356.0,United States of America,2016,1392,telecommunications
3,249164402,{'com.linkedin.voyager.deco.jobs.web.shared.We...,Data Scientist,"Rational is a full service, integrated agency ...",False,"Seattle, WA",NaN,242.0,United States of America,2016,805290,advertising services
4,249164246,{'com.linkedin.voyager.deco.jobs.web.shared.We...,Data Analyst - Cisco expereince a must,"Data Analyst - Finances & Operations Raleigh, ...",False,"Raleigh, NC",NaN,353.0,United States of America,2016,60921,it services and it consulting


In [69]:
df_with_industries['Industry'].value_counts()

Industry
it services and it consulting          11569
staffing and recruiting                 9767
software development                    7169
financial services                      2171
business consulting and services        1523
                                       ...  
political organizations                    1
animation and post-production              1
museums                                    1
leasing non-residential real estate        1
physicians                                 1
Name: count, Length: 162, dtype: int64

In [70]:
df_with_industries[df_with_industries["Industry"].isnull()]['year'].value_counts()

year
2016    9031
2018    1172
Name: count, dtype: int64

In [71]:
df_with_industries_not_found = df_with_industries[df_with_industries["Industry"].isnull()]
df_with_industries_not_found = df_with_industries_not_found[['Company ID']].drop_duplicates()
df_with_industries_not_found.head()

,Company ID
19,1185607
29,17979372
40,825918
41,4843353
50,2821280


In [72]:
len(df_with_industries_not_found)

4318

In [29]:
# df_with_industries_not_found.to_csv("C:\\Users\\eshad\\OneDrive\\Desktop\\TU Delft\\Thesis\\Web Scrape PoC\\Data\\4. Filtered preprocessed data with industry\\industries_not_found_batch_3.csv")

## Saving Data

In [151]:
industry_df_no_dupes_valid_data.to_csv("C:\\Users\\eshad\\OneDrive\\Desktop\\TU Delft\\Thesis\\Web Scrape PoC\\Data\\4. Filtered preprocessed data with industry\\all_data_with_industries\\final_industry_list.csv")